In [57]:
import os
import time

import polars as pl
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer, StandardScaler

In [58]:
SEED = 462

In [59]:
def load_data(filename):
    path = os.path.join("data", "tabular", filename)
    data = pl.read_csv(path).to_numpy()
    X = data[:, :-1].astype(float)
    y = data[:, -1]
    return X, y

In [60]:
X_train, y_train = load_data("train_processed.csv")
X_val, y_val = load_data("validation_processed.csv")
X_test, y_test = load_data("test_processed.csv")

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [61]:
print("Starting Training...")
start_time = time.time()

model = OneVsRestClassifier(LogisticRegression(solver="lbfgs", max_iter=10000, random_state=SEED))
model.fit(X_train, y_train)

end_time = time.time()
print(f"Total Training Time: {end_time - start_time:.4f} seconds\n")

Starting Training...
Total Training Time: 0.0338 seconds



In [62]:
def print_metrics(X, y, name):
    pred = model.predict(X)
    probs = model.predict_proba(X)

    acc = accuracy_score(y, pred)
    prec = precision_score(y, pred, average="weighted")
    rec = recall_score(y, pred, average="weighted")
    f1 = f1_score(y, pred, average="weighted")

    lb = LabelBinarizer()
    lb.fit(y_train)
    y_bin = lb.transform(y)
    auc = roc_auc_score(y_bin, probs, multi_class="ovr", average="weighted")

    print(f"--- {name} Metrics ---")
    print(f"Accuracy:  {acc * 100:.2f}%")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print(f"AUC (Avg): {auc:.4f}\n")

In [63]:
print_metrics(X_train, y_train, "Training")
print_metrics(X_val, y_val, "Validation")
print_metrics(X_test, y_test, "Test")

--- Training Metrics ---
Accuracy:  85.23%
Precision: 0.8627
Recall:    0.8523
F1 Score:  0.8492
AUC (Avg): 0.9850

--- Validation Metrics ---
Accuracy:  84.80%
Precision: 0.8633
Recall:    0.8480
F1 Score:  0.8460
AUC (Avg): 0.9854

--- Test Metrics ---
Accuracy:  90.03%
Precision: 0.9041
Recall:    0.9003
F1 Score:  0.8981
AUC (Avg): 0.9878

